In [3]:
import numpy as np
import pandas as pd
import re
# import re
from os.path import join
import os
import glob

from tools import *

input_prefix="/data/liu/mimic3/CLAMP_NER/input"
output_prefix="/data/liu/mimic3/CLAMP_NER/ner-attribute/output"
save_csv_prefix="/data/liu/mimic3/CLAMP_NER/ner-attribute/output_csv"

In [6]:
## Import CLAMP NER result
def splitfilename(filename):
    ids= filename.rsplit(".",1)[0].split("_")
    return ids

def readtxt(filename):
    text=""
    with open(filename,'r') as myfile:
        text=myfile.read()
    return text


# def files2df(folder):
#     os.chdir(join(output_prefix,folder))
#     output_data = [getfileinfo(f) for f in glob.glob("*.txt") if (os.path.isfile(f))]
#     return output_data
                                                   
def parseNErow(row):

    indexes=row[1:3]
    attrs=[attr.split('=')[1] for attr in row[3:] if ("=" in attr)]
    return indexes+attrs

# def parseNEtext(text,num_cols=8, 
# #                 cols=["START INDEX","END INDEX","SEMANTIC","ASSERTION","CUI","NE"]
#                cols=["START INDEX","END INDEX","SEMANTIC","ASSERTION","CUI","sentProb","conceptProb","NE"]):
#     df = pd.DataFrame([parseNErow(x.split('\t')) for x in text.split('\n')])
#     if(len(df.columns)==num_cols): 
#         df.columns=cols
#         return df
#     else: return None
    
def parseNEtext(text,num_cols=8, 
#                 cols=["START INDEX","END INDEX","SEMANTIC","ASSERTION","CUI","NE"]
               cols=["START INDEX","END INDEX","SEMANTIC","ASSERTION","CUI","sentProb","conceptProb","NE"]):
    df = pd.DataFrame([x.split('\t') for x in text.split('\n')])
    return df
#     if(len(df.columns)==num_cols): 
#         df.columns=cols
#         return df
#     else: return None

def savetext2csv(folder, write=True):
    os.chdir(join(output_prefix,folder))
    files = [f for f in glob.glob("*.txt") if (os.path.isfile(f))]

    li=[]
    for file in files:
        text=readtxt(file)

        df = parseNEtext(text.split("\nSentence",1)[0])
        if(not df is None):
            li.append((file, df))
            if(write):
                filename=file.replace(".txt","")
                write2file(df,join(save_csv_prefix,folder,filename))
    return li


def addidcolumns(file, df):
    idcols=['SUBJECT_ID','HADM_ID','GROUP_RANK']
    ids = splitfilename(file)
    for i in range(3):
        df.insert(i, idcols[i], ids[i])
    return df

def concatcsvs(folder,write=False):
    csvdf = savetext2csv(folder,write)
    concatcsv = [addidcolumns(file, df) for file, df in csvdf]
    concateddf = pd.concat(concatcsv, axis=0)
    if(write):
        write2file(concateddf,join(save_csv_prefix,folder,"ALL/ALL"))
    return concateddf
    

In [100]:
def getdisease(df):
    diseasedf = df[(df[3]=="semantic=problem") & (df[5].str.contains("SNOMEDCT_US"))].drop(columns=[0]).iloc[:,0:11]
    cols=["SUBJECT_ID", "HADM_ID", "GROUP_RANK","START INDEX","END INDEX","SEMANTIC","ASSERTION","CUI","sentProb","conceptProb","NE"]
    diseasedf.columns=cols
    diseasedf = diseasedf.reset_index()
    
    diseasedf["CUI"], diseasedf["SNOMEDCT_US"] = diseasedf.CUI.str.split(" ",1).str
#     diseasedf=diseasedf.dropna(subset=["SNOMEDCT_US"])
    diseasedf["SNOMEDCT_US"]=diseasedf["SNOMEDCT_US"].apply(lambda x: x[x.find("[")+1:-1])
    return diseasedf

In [101]:
for folder in ["hospital_course"]:
    concatcsv = concatcsvs(folder,False)
    disease_df = getdisease(concatcsv)
    write2file(disease_df,join(save_csv_prefix,folder,"ALL/ALL_Disease"))

/home/liu/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:70: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [102]:
disease_df.head()

,index,SUBJECT_ID,HADM_ID,GROUP_RANK,START INDEX,END INDEX,SEMANTIC,ASSERTION,CUI,sentProb,conceptProb,NE,SNOMEDCT_US
0,4,6702,143782,1,111,127,semantic=problem,assertion=present,cui=C0032227,sentProb=0.825,conceptProb=1.0,ne=pleural effusion,60046008
1,7,6702,143782,1,172,186,semantic=problem,assertion=present,cui=C0001418,sentProb=0.524,conceptProb=0.996,ne=adenocarcinoma,"443961001, 35917007"
2,21,20630,161386,1,211,222,semantic=problem,assertion=absent,cui=C0702249,sentProb=0.992,conceptProb=0.999,ne=infiltrates,231287002
3,22,20630,161386,1,226,235,semantic=problem,assertion=absent,cui=C0013687,sentProb=0.992,conceptProb=0.993,ne=effusions,41699000
4,3,17496,127002,1,105,113,semantic=problem,assertion=present,cui=C0277797,sentProb=0.677,conceptProb=0.678,ne=afebrile,86699002


In [16]:
# concatcsv.SEMANTIC.unique()
concatcsv=read_data(join(save_csv_prefix,"medications_on_admission/ALL/ALL"))

In [17]:
concatcsv.SEMANTIC.unique()

array(['treatment', 'drug', 'problem', 'BDL', 'NEG', 'test', 'BDL_Of',
       'NEG_Of', 'COU', 'drug::STRENGTH', 'drug::RUT', 'COU_Of', 'attrOf',
       'labvalue', 'valueOf', 'UNC', 'drug::FREQ', 'SEV', 'drug::NEC',
       'drug::DRT', 'UNC_Of', 'SEV_Of', 'drug::FORM', 'CON', 'CON_Of',
       'SUB', 'SUB_Of', 'DDF-DIN', 'DDF-DOSEUNIT-RUT', 'drug::DOSEAMT',
       'DOSEUNIT-DIN', 'DDF-DOSEUNIT-DPN', 'DBN-DPN-DIN', 'RUT-DPN-DIN',
       'DDF-DOSEUNIT-DIN', 'DPN-DIN'], dtype=object)

In [14]:
concatcsv.SEMANTIC.unique()

array(['treatment', 'drug', 'problem', 'BDL', 'NEG', 'test', 'BDL_Of',
       'NEG_Of', 'COU', 'drug::STRENGTH', 'drug::RUT', 'COU_Of', 'attrOf',
       'labvalue', 'valueOf', 'UNC', 'drug::FREQ', 'SEV', 'drug::NEC',
       'drug::DRT', 'UNC_Of', 'SEV_Of', 'drug::FORM', 'CON', 'CON_Of',
       'SUB', 'SUB_Of', 'DDF-DIN', 'DDF-DOSEUNIT-RUT', 'drug::DOSEAMT',
       'DOSEUNIT-DIN', 'DDF-DOSEUNIT-DPN', 'DBN-DPN-DIN', 'RUT-DPN-DIN',
       'DDF-DOSEUNIT-DIN', 'DPN-DIN'], dtype=object)

## Create Folders for NER ATTR

In [1]:
## create folder
import os

# define the name of the directory to be created
prefix = "/data/liu/mimic3/CLAMP_NER/ner-attribute"
output=join(prefix,"output")
output_csv=join(prefix,"output_csv")

def create_folder(path):
    try:
        os.mkdir(path)
    except OSError:
        print ("Creation of the directory %s failed" % path)
    else:
        print ("Successfully created the directory %s " % path)
        
create_folder(output)
create_folder(output_csv)
for subfolder in ["hospital_course","medical_history","medications_on_admission"]:
    create_folder(join(output,subfolder))
    create_folder(join(output_csv,subfolder))
#     create_folder(join(output_csv,subfolder,"ALL"))
    
## if want to delete folder
# import shutil
# shutil.rmtree("path_to_dir")

## RXNORM CUI -> NDC

In [26]:
mapping_prefix = "/data/liu/mimic3/MAPPING/"
processed_map_prefix="/data/liu/mimic3/MAPPING/PROCESSED/"

In [22]:
rxnorm_ndc_df = pd.read_csv(join(mapping_prefix,"RXNSAT.RRF"),sep="|",header=None)


In [24]:
rxnorm_ndc_df=rxnorm_ndc_df.iloc[:,[3,4,5,8,9,10]]
cols=["CUI_CODE","CAT","CUI_SUBCODE","CODE_TYPE","SOURCE","CODE"]
rxnorm_ndc_df.columns = cols
rxnorm_ndc_df.head()

,CUI_CODE,CAT,CUI_SUBCODE,CODE_TYPE,SOURCE,CODE
0,8717795,AUI,58488005,UMLSAUI,RXNORM,A27769867
1,8717795,AUI,58488005,UMLSCUI,RXNORM,C0000052
2,8717796,AUI,58488005,UMLSAUI,RXNORM,A27780666
3,8717796,AUI,58488005,UMLSCUI,RXNORM,C0000052
4,8717808,AUI,58488005,UMLSAUI,RXNORM,A27766654


In [25]:
rxnorm_ndc_df = rxnorm_ndc_df[(rxnorm_ndc_df["CODE_TYPE"]=="NDC") & (rxnorm_ndc_df["SOURCE"]=="RXNORM")]
rxnorm_ndc_df.head()

,CUI_CODE,CAT,CUI_SUBCODE,CODE_TYPE,SOURCE,CODE
846934,707001,AUI,91348,NDC,RXNORM,51927275500
846989,707028,AUI,91349,NDC,RXNORM,00295117904
846990,707028,AUI,91349,NDC,RXNORM,00295117916
846991,707028,AUI,91349,NDC,RXNORM,00363026708
846992,707028,AUI,91349,NDC,RXNORM,00363026816


In [27]:
write2file(rxnorm_ndc_df,join(processed_map_prefix,"rxnorm_ndc_df"))

## MAP DISEASE SNOMED_CID -> ICD9

In [104]:
import numpy as np
import pandas as pd
import re
# import re
from os.path import join

from tools import *

# from sklearn.decomposition import LatentDirichletAllocation
# from scipy.spatial import distance

# import matplotlib.pyplot as plt

## rename EXTRACT_FEATURES to DIMENSION_REDUCTION
read_prefix = "/data/MIMIC3/"
mapping_prefix = "/data/liu/mimic3/MAPPING/"

In [103]:
# icd9_SNOMED_df2 =pd.read_csv(join(mapping_prefix,'ICD9CM_SNOMED_MAP_1TOM_201912.txt')
#                          ,sep='\t', dtype={'ICD9_CODE':str, 'SNOMED_CID':str}).dropna(subset=['SNOMED_CID'])
# icd9_SNOMED_df2.head()

In [107]:
icd9_SNOMED_df =pd.read_csv(join(mapping_prefix,'ICD9CM_SNOMED_MAP_1TO1_201912.txt')
                         ,sep='\t', dtype={'ICD9_CODE':str, 'SNOMED_CID':str}).dropna(subset=['SNOMED_CID'])

icd9_SNOMED_df = icd9_SNOMED_df[["ICD_CODE","SNOMED_CID"]]
icd9_SNOMED_df.head()

,ICD_CODE,SNOMED_CID
0,427.31,49436004
1,599.0,68566005
2,486,233604007
3,585.6,46177005
4,414.01,1641000119107


In [111]:
folder="hospital_course"
disease_df = read_data(join(save_csv_prefix,folder,"ALL/ALL_Disease")).drop(columns=["index"])
disease_df.head()

,SUBJECT_ID,HADM_ID,GROUP_RANK,START INDEX,END INDEX,SEMANTIC,ASSERTION,CUI,sentProb,conceptProb,NE,SNOMEDCT_US
0,6702,143782,1,111,127,semantic=problem,assertion=present,cui=C0032227,sentProb=0.825,conceptProb=1.0,ne=pleural effusion,60046008
1,6702,143782,1,172,186,semantic=problem,assertion=present,cui=C0001418,sentProb=0.524,conceptProb=0.996,ne=adenocarcinoma,"443961001, 35917007"
2,20630,161386,1,211,222,semantic=problem,assertion=absent,cui=C0702249,sentProb=0.992,conceptProb=0.999,ne=infiltrates,231287002
3,20630,161386,1,226,235,semantic=problem,assertion=absent,cui=C0013687,sentProb=0.992,conceptProb=0.993,ne=effusions,41699000
4,17496,127002,1,105,113,semantic=problem,assertion=present,cui=C0277797,sentProb=0.677,conceptProb=0.678,ne=afebrile,86699002


In [109]:
icd9_SNOMED_map = icd9_SNOMED_df.set_index('SNOMED_CID')['ICD_CODE'].to_dict()
icd9_SNOMED_map.get("49436004", None)

'427.31'

In [115]:
disease_df["SNOMEDCT_US"].head().apply(lambda x: [icd9_SNOMED_map.get(key, None) for key in x.strip().split(',')])

0         [511.9]
1    [None, None]
2          [None]
3          [None]
4          [None]
Name: SNOMEDCT_US, dtype: object

In [16]:
icd9_SNOMED_df[icd9_SNOMED_df['SNOMED_CID']=='4834000']

,ICD_CODE,ICD_NAME,IS_CURRENT_ICD,IP_USAGE,OP_USAGE,AVG_USAGE,IS_NEC,SNOMED_CID,SNOMED_FSN,IS_1-1MAP,CORE_USAGE,IN_CORE
5807,002.0,Typhoid fever,1,0.00004,NaN,0.00002,0,4834000,Typhoid fever (disorder),1,0.0005,1


In [6]:
# concatcsv.SEMANTIC.unique()
medical_history_df=read_data(join(save_csv_prefix,"medical_history/ALL/ALL"))
medical_history_df.head()

,SUBJECT_ID,HADM_ID,GROUP_RANK,START INDEX,END INDEX,SEMANTIC,ASSERTION,CUI,sentProb,conceptProb,NE
0,19981,159283,2,21,39,problem,present,C0018834 SNOMEDCT_US[16331000],0.998,1.0,heartburn symptoms
1,19981,159283,2,72,98,problem,present,0.971,0.971,additional epigastric pain,NaN
2,19981,159283,2,83,93,BDL,epigastric,NaN,NaN,NaN,NaN
3,19981,159283,2,135,146,problem,present,0.994,0.995,a dull ache,NaN
4,19981,159283,2,149,152,test,present,C0079304,0.866,0.995,EGD
